# Language-based generative models

LSTM and GPT-based vanila chemical language models have been implemented and simple usage is explained. Development history and statistics for parameter selection are reported separately in `lstm_generative_model_flow.md`.

Currently, LSTM-based model is stable and should be used for language modeling. The GPT-based model is okay, but still hard to understand its behaivors in particular hyperparameters (nblocks, nheads, etc).

In [2]:
import sys
sys.path.append('../')
from src.paths import ensure_dirs, PRETRAIN_DATA, PRETRAIN_RESULTS, FINETUNE_FILTER
ensure_dirs()
print(f'Fine-tuning data status: {FINETUNE_FILTER}')

Fine-tuning data status: filtered


## 0. Function to run command line application on a jupyter notebook

In [2]:
def importstr(module_str, from_=None):
	"""
	module_str: module to be loaded as string 
	>>> importstr('os) -> <module 'os'>
	"""
	if (from_ is None) and ':' in module_str:
		module_str, from_ = module_str.rsplit(':')
	module = __import__(module_str)
	for sub_str in module_str.split('.')[1:]:
		module = getattr(module, sub_str)
	
	if from_:
		try:
			return getattr(module, from_)
		except:
			raise ImportError(f'{module_str}.{from_}')
	return module


def run(app, *argv):
	argv=list(argv)
	app_cls=importstr(*app.rsplit('.',1))
	app_cls(argv).main()

## 1. Tokenize SMILES
To determine which tokens are used ('[C@H]', 'c', '8', etc.), a data set is tokenized to calculate the frequency of tokens. This process can be skipped if you already know the tokens to be used for your generative models.

### 1. Calculate frequency of tokens
To calcualte the frequency of tokens, `apps.count_token_frequency.py` is used. Example data set is provided as well in the data folder.



If you want to run the script as a command line program. Use the following codes. If you want to run on a notebook, run the codes in the next cell.
```
$ python generative_models/apps/count_token.py --data data/top2000_curated_cpds_chembl31.tsv --smi-colname washed_openeye_smiles --outdir tokenize_results --heavy-atom-ratio-thres 0.95
```

In [ ]:
dataset_list = ['pubchem_filtered_ac', 
                'pubchem_unfiltered_ac', 
                'pubchem_inac', 
                'chembl_filtered', 
                'chembl_unfiltered', 
                'zinc']

In [ ]:
import sys 
sys.path.append('../src/model/')

for dataset in dataset_list:
	run("generative_models.apps.CountTokens.CountTokenFreqApp",
		f"--data={PRETRAIN_DATA}/{dataset}_rdsmi3.tsv",
		"--smi-colname=rdkit_smiles",
		f"--outdir={PRETRAIN_RESULTS}/{dataset}_results/tokenize_results",
		"--heavy-atom-ratio-thres=0.95")

Start logging
Loaded molecules: 125149


cannnot handeld mols in rdkit: 0
passed mols: 125149
maximum heavy atom number is 128
minimum heavy atom number is 5
the NHA percentile 0.95
the NHA threshold 38.0
selected molecules: 119665


Then, you can find **tokenize_resluts** folder specficed as input arguments. Looking at token frequencies in `token_requency.tsv`, you can determine which tokens you are going to use for a subsequent analysis, as follows.

In [5]:
import pandas as pd 
freq = pd.read_csv(f'{PRETRAIN_RESULTS}/{dataset}_results/tokenize_results/token_frequency.tsv', sep='\t', index_col=0)
print(freq)


        frequency
c         1342612
C          898109
(          510468
)          510468
1          352208
O          325534
2          260868
=          239737
N          201959
n          135041
3          118084
S           39204
F           34531
-           28161
4           27192
Cl          23110
s           19913
/           18058
o           15958
[C@H]       15689
[C@@H]      15345
[nH]        11740
[N+]         7766
[O-]         7745
#            7281
Br           6537
\            4300
5            2970
I             767
[C@@]         516
[C@]          501
P             342
[n+]          306
6             198
7              18
[c-]           15
[S@@]           9
B               6
[S+]            6
[P+]            3
[s+]            3
[As]            3


There is a gap between `[+]` and `6` as below:
|Token| Frequency|
|----|----|
|[S+]|183|
|[n+]|178|
|6|44|

Thus, tokens with more than 100 frequencies are employed here.


In [6]:
ok_tokens = freq[freq['frequency'] > 100].index
ok_tokens

Index(['c', 'C', '(', ')', '1', 'O', '2', '=', 'N', 'n', '3', 'S', 'F', '-',
       '4', 'Cl', 's', '/', 'o', '[C@H]', '[C@@H]', '[nH]', '[N+]', '[O-]',
       '#', 'Br', '\', '5', 'I', '[C@@]', '[C@]', 'P', '[n+]', '6'],
      dtype='object')

### 2. Select eligible molecules and vocabulary set (dictionarr) for generation
We decided the tokens eligible for generative models. The training data set `ok_mols.tsv` were further filtered based on the selected tokens above.
`app.select_mols_vocab.py` is conducted.

In [ ]:
import sys 
sys.path.append('../src/model/')

for dataset in dataset_list:
	appname  = 'generative_models.apps.SelectMolsVocab.SelectMolsVocabApp'
	run(appname,
		f"--data={PRETRAIN_RESULTS}/{dataset}_results/tokenize_results/ok_mols.tsv",
		"--smi-colname=rdkit_smiles",
		f"--tokens={PRETRAIN_RESULTS}/{dataset}_results/tokenize_results/token_frequency.tsv",
		f"--outdir={PRETRAIN_RESULTS}/{dataset}_results/dataset4lstm",
		"--token-frequency-thres=50")

Start logging
Loaded molecules: 119665
curating tokens: frequency threshold: 50
ok tokens: 34
Index(['c', 'C', '(', ')', '1', 'O', '2', '=', 'N', 'n', '3', 'S', 'F', '-',
       '4', 'Cl', 's', '/', 'o', '[C@H]', '[C@@H]', '[nH]', '[N+]', '[O-]',
       '#', 'Br', '\', '5', 'I', '[C@@]', '[C@]', 'P', '[n+]', '6'],
      dtype='object')
eligible smiles: 119623


/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/noteboooks/../src/model/generative_models/apps/SelectMolsVocab.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  okmols['ntokens_srt_end'] = okmols['rdkit_smiles'].apply(lambda x: count_moltokens(x, include_begin_and_end=True))


You can find `dataset4lstm` folder, where passed_molecules tsv and vocaburary dictionary pickle files are stored. These two files are used for training LSTM model.

## 2 Build LSTM model
An LSTM model is trained on the data set specified. Various options are available. You can decide appropriate one by traial, or you can read the report on benchmark testing regarding hyperparameteres.

In [ ]:
import sys 
sys.path.append('../src/model/')

for dataset in dataset_list:
	dataset_dir = f'{PRETRAIN_RESULTS}/{dataset}_results/dataset4lstm'

	appname  	= 'generative_models.apps.BuildModel.TrainerApp'
	nworkers 	= 20
	data 	   	= f'{dataset_dir}/passed_mols.tsv'
	vocab 		= f'{dataset_dir}/vocab.pickle'
	outfd 		= f'{PRETRAIN_RESULTS}/{dataset}_results/vanilalstm'
	smicol     	= 'rdkit_smiles'
	valratio 	= 0.1
	espatience 	= 4
	epochs 		= 30
	nsamples 	= 1000
	batch_size  = 128
	save_snapshot = '10,20,30'
	# model parameters 
	model 		= 'lstm'
	lr 			= 0.0005
	embed		= 256
	nlayers 	= 4
	hidden  	= 512
	dropout 	= 0.2
	layernorm 	= True


	general_params = [
				f'--num-workers={nworkers}',
				'--override-folder=1',
				f'--epochs={epochs}', 
				f'--validation-ratio={valratio}',
				'--exclude-pad-loss=1',
				f'--early-stopping-patience={espatience}',
				f'--data={data}',
				f'--smi-colname={smicol}',
				f'--vocab={vocab}',
				f'--outdir={outfd}',
				f'--sampling-epoch={nsamples}',
				f'--batch-size={batch_size}',
				f'--save-snapshot-models={save_snapshot}'
				]

	model_params = [
		f'--model={model}',
		f'--lr={lr}',
		f'--embed-dim={embed}',
		f'--dropout-ratio={dropout}',
		f'--nlayers={nlayers}',
		f'--hidden-dim={hidden}',
		'--layernorm' if layernorm else '--no-layernorm'
	]
	params = general_params + model_params
	run(appname, *params)


lstm
Start logging
CUDA is on: 1 devices
Model initialization is done
Starting: GenerativeModelTrainer, Namespace(num_workers=20, outdir='/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/results/pretrain/pubchem_inac_results/vanilalstm', case='', override_folder=1, tensorboard_prefix='tensor_board', data='/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/results/pretrain/pubchem_inac_results/dataset4lstm/passed_mols.tsv', smi_colname='rdkit_smiles', vocab='/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/results/pretrain/pubchem_inac_results/dataset4lstm/vocab.pickle', random_seed=42, debug=None, sampling_epoch=1000, model='lstm', write_xlsx=None, save_snapshot_models='10,20,30', batch_size=128, epochs=30, validation_ratio=0.1, lr=0.0005, exclude_pad_loss=1, early_stopping_patience=4, load_model=None, model_structure=None, model_state=None, standardize_smiles=None, embed_dim=256, dropout_ratio=0.2, nlayers=4, hidden_dim=512, layer

[19:03:55] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 18
[19:03:55] SMILES Parse Error: extra open parentheses for input: 'Cc1ccc(NC1Cn2cc(NC(=O)Nc3ccc(OC(F)(F)F)cc3F)nc3nc2s1'
[19:03:55] SMILES Parse Error: unclosed ring for input: 'CC1(C)C(C)C(C)N2CCCCN(C(=O)NCc3ccccc3)C1'
[19:03:55] Explicit valence for atom # 9 C, 5, is greater than permitted
[19:03:55] SMILES Parse Error: unclosed ring for input: 'COC(=O)C1/C(=N/NC2=O)c(=O)oc1N'
[19:03:55] Can't kekulize mol.  Unkekulized atoms: 1 2 20 21 22 23 24 25 26
[19:03:55] SMILES Parse Error: unclosed ring for input: 'CCCCC(=O)Nc1cccc(-n2nc(=S)n(C3CC(=O)N(C)C)(C(=O)N(C)C)C2)c1'
[19:03:55] Can't kekulize mol.  Unkekulized atoms: 19 20 21 22 23
[19:03:55] SMILES Parse Error: unclosed ring for input: 'Cn1c(S(=O)(=O)Nc2ccc(F)cc2)nc2c1ccccc12'
[19:03:55] SMILES Parse Error: unclosed ring for input: 'COc1ccc(C(=O)Nc2ccc([N+](=O)[O-])c3nccn22)cc1'
[19:03:55] SMILES Parse Error: unclosed ring for input: 'COc1cc(-c2cc(C(=O)NCc3ccccc3)sn2c2CNC(

Epoch 2 -- Batch 1/ 842, training loss 0.6875993013381958
Epoch 2 -- Batch 2/ 842, training loss 0.7156004309654236
Epoch 2 -- Batch 3/ 842, training loss 0.6980999112129211
Epoch 2 -- Batch 4/ 842, training loss 0.7139374017715454
Epoch 2 -- Batch 5/ 842, training loss 0.6910369992256165
Epoch 2 -- Batch 6/ 842, training loss 0.7170813083648682
Epoch 2 -- Batch 7/ 842, training loss 0.7075843214988708
Epoch 2 -- Batch 8/ 842, training loss 0.7003893852233887
Epoch 2 -- Batch 9/ 842, training loss 0.7086516618728638
Epoch 2 -- Batch 10/ 842, training loss 0.7113326787948608
Epoch 2 -- Batch 11/ 842, training loss 0.6944671869277954
Epoch 2 -- Batch 12/ 842, training loss 0.727820634841919
Epoch 2 -- Batch 13/ 842, training loss 0.6805467009544373
Epoch 2 -- Batch 14/ 842, training loss 0.7049128413200378
Epoch 2 -- Batch 15/ 842, training loss 0.6942419409751892
Epoch 2 -- Batch 16/ 842, training loss 0.7016972303390503
Epoch 2 -- Batch 17/ 842, training loss 0.6909859776496887
Epoch 2

[19:04:05] SMILES Parse Error: syntax error while parsing: Cc1ccc(NC(=O)COc2c(C)[nH]c(=O)c3cc(-c4ccc(Cl)cc4)nc(4)c23)cc1
[19:04:05] SMILES Parse Error: Failed parsing SMILES 'Cc1ccc(NC(=O)COc2c(C)[nH]c(=O)c3cc(-c4ccc(Cl)cc4)nc(4)c23)cc1' for input: 'Cc1ccc(NC(=O)COc2c(C)[nH]c(=O)c3cc(-c4ccc(Cl)cc4)nc(4)c23)cc1'
[19:04:05] Can't kekulize mol.  Unkekulized atoms: 1 2 4 5 6 19 20
[19:04:05] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10
[19:04:05] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 10 12 13 14 15 16
[19:04:05] SMILES Parse Error: unclosed ring for input: 'CC(C)=CCn1c(=S)sc2nc3c(cc22)CCCC3'
[19:04:05] SMILES Parse Error: unclosed ring for input: 'Cc1cccc2c(c1)ccc(C(=O)NC(=O)C2CC2)c1C'
[19:04:05] SMILES Parse Error: extra close parentheses while parsing: Cc1ccc2[nH]c(=O)c(SCC(=O)N3CCc4ccccc43)n2)cc1
[19:04:05] SMILES Parse Error: Failed parsing SMILES 'Cc1ccc2[nH]c(=O)c(SCC(=O)N3CCc4ccccc43)n2)cc1' for input: 'Cc1ccc2[nH]c(=O)c(SCC(=O)N3CCc4ccccc43)n2)cc1'
[19:04:05] 

Epoch 3 -- Batch 1/ 842, training loss 0.5862991213798523
Epoch 3 -- Batch 2/ 842, training loss 0.6026047468185425
Epoch 3 -- Batch 3/ 842, training loss 0.6008363366127014
Epoch 3 -- Batch 4/ 842, training loss 0.6324578523635864
Epoch 3 -- Batch 5/ 842, training loss 0.5863271355628967
Epoch 3 -- Batch 6/ 842, training loss 0.6186871528625488
Epoch 3 -- Batch 7/ 842, training loss 0.6066606640815735
Epoch 3 -- Batch 8/ 842, training loss 0.6082261800765991
Epoch 3 -- Batch 9/ 842, training loss 0.6186850070953369
Epoch 3 -- Batch 10/ 842, training loss 0.6124225854873657
Epoch 3 -- Batch 11/ 842, training loss 0.6059475541114807
Epoch 3 -- Batch 12/ 842, training loss 0.5809412598609924
Epoch 3 -- Batch 13/ 842, training loss 0.5831202864646912
Epoch 3 -- Batch 14/ 842, training loss 0.6151729822158813
Epoch 3 -- Batch 15/ 842, training loss 0.5801334977149963
Epoch 3 -- Batch 16/ 842, training loss 0.6074866056442261
Epoch 3 -- Batch 17/ 842, training loss 0.5938974618911743
Epoch 

...

[19:08:33] Can't kekulize mol.  Unkekulized atoms: 9 10 11 18 20
[19:08:33] SMILES Parse Error: unclosed ring for input: 'Cn1c(=O)c2c3c(ncn2CC(=O)OCC(=O)Nc2ccc(Br)cc2)n(C)c1=O'
[19:08:33] SMILES Parse Error: unclosed ring for input: 'CCOC(=O)C1(O)C2CCCCC2C1(O)C(=O)N(C)C2CCCCC1'
[19:08:33] Explicit valence for atom # 12 O, 3, is greater than permitted
[19:08:33] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 12 27
[19:08:33] SMILES Parse Error: unclosed ring for input: 'c1ccc(COC2C(C3COC4(CCCCC4)O3)OC3OC4(CCCCC4)OC3[C@H]2C(=O)N(CCC(F)(F)F)C3=O)cc1'
[19:08:33] Explicit valence for atom # 12 O, 3, is greater than permitted
[19:08:33] SMILES Parse Error: syntax error while parsing: Cc1cccc(-n2nc3c(c2NC(=O)Cc2ccc(O)c(OC)c2)CS(=O)==)C3)c1C
[19:08:33] SMILES Parse Error: Failed parsing SMILES 'Cc1cccc(-n2nc3c(c2NC(=O)Cc2ccc(O)c(OC)c2)CS(=O)==)C3)c1C' for input: 'Cc1cccc(-n2nc3c(c2NC(=O)Cc2ccc(O)c(OC)c2)CS(=O)==)C3)c1C'
[19:08:33] Explicit valence for atom # 9 C, 5, is greater than permitted

Epoch 29 -- Batch 1/ 842, training loss 0.295406311750412
Epoch 29 -- Batch 2/ 842, training loss 0.3022112548351288
Epoch 29 -- Batch 3/ 842, training loss 0.29400551319122314
Epoch 29 -- Batch 4/ 842, training loss 0.30828857421875
Epoch 29 -- Batch 5/ 842, training loss 0.3039529323577881
Epoch 29 -- Batch 6/ 842, training loss 0.29846230149269104
Epoch 29 -- Batch 7/ 842, training loss 0.30116215348243713
Epoch 29 -- Batch 8/ 842, training loss 0.3059957027435303
Epoch 29 -- Batch 9/ 842, training loss 0.30876341462135315
Epoch 29 -- Batch 10/ 842, training loss 0.3003529906272888
Epoch 29 -- Batch 11/ 842, training loss 0.30735528469085693
Epoch 29 -- Batch 12/ 842, training loss 0.3174561560153961
Epoch 29 -- Batch 13/ 842, training loss 0.30606985092163086
Epoch 29 -- Batch 14/ 842, training loss 0.3028392791748047
Epoch 29 -- Batch 15/ 842, training loss 0.29791346192359924
Epoch 29 -- Batch 16/ 842, training loss 0.29539093375205994
Epoch 29 -- Batch 17/ 842, training loss 0.3

[19:08:43] Explicit valence for atom # 11 C, 5, is greater than permitted
[19:08:43] Can't kekulize mol.  Unkekulized atoms: 6 7 8
[19:08:43] SMILES Parse Error: unclosed ring for input: 'O=C(O)C1C2CC3C1C(=O)Oc1ccccc1C1c3sc(=O)[nH]c2SC31'
[19:08:43] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20
[19:08:43] SMILES Parse Error: unclosed ring for input: 'CCN(CC)CC(=O)N1CCc2c(sc3ccccc23)C12CC1'
[19:08:43] SMILES Parse Error: unclosed ring for input: 'COc1ccc2c(c1)sc1nc(C(C)=O)c(=O)n2C'
[19:08:43] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 7 8 9 10 11 12 13 14
[19:08:43] SMILES Parse Error: ring closure 2 duplicates bond between atom 14 and atom 15 for input: 'Cc1ccc(CN(C(=O)c2csnn2)C2C2CCCCC2)c(C)c1'
[19:08:43] SMILES Parse Error: unclosed ring for input: 'CCOC(=O)c1c(C)oc2ccc(N(C(C)=O)S(=O)(=O)c3ccc4c5c(ccc3c3)CCC4)cc12'
[19:08:43] SMILES Parse Error: unclosed ring for input: 'CN1C(C(=O)O)C2C3C=CC(C4)C2C1=O'
[19:08:43] SMILES Parse Error: ring closure 4 duplicates bond betw

Epoch 30 -- Batch 1/ 842, training loss 0.2970809042453766
Epoch 30 -- Batch 2/ 842, training loss 0.3127555847167969
Epoch 30 -- Batch 3/ 842, training loss 0.29363590478897095
Epoch 30 -- Batch 4/ 842, training loss 0.301332950592041
Epoch 30 -- Batch 5/ 842, training loss 0.3112838566303253
Epoch 30 -- Batch 6/ 842, training loss 0.2928890287876129
Epoch 30 -- Batch 7/ 842, training loss 0.3063929080963135
Epoch 30 -- Batch 8/ 842, training loss 0.3075919449329376
Epoch 30 -- Batch 9/ 842, training loss 0.2963351905345917
Epoch 30 -- Batch 10/ 842, training loss 0.29250746965408325
Epoch 30 -- Batch 11/ 842, training loss 0.29490116238594055
Epoch 30 -- Batch 12/ 842, training loss 0.3009416162967682
Epoch 30 -- Batch 13/ 842, training loss 0.30367615818977356
Epoch 30 -- Batch 14/ 842, training loss 0.3042241632938385
Epoch 30 -- Batch 15/ 842, training loss 0.3034258186817169
Epoch 30 -- Batch 16/ 842, training loss 0.3045646548271179
Epoch 30 -- Batch 17/ 842, training loss 0.298

[19:08:54] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 12 13 21
[19:08:54] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 22
[19:08:54] SMILES Parse Error: extra close parentheses while parsing: Cc1ncn(-c2ccc(Nc3cc(-c4nocc4C)ccn3)cc2)n1)c1
[19:08:54] SMILES Parse Error: Failed parsing SMILES 'Cc1ncn(-c2ccc(Nc3cc(-c4nocc4C)ccn3)cc2)n1)c1' for input: 'Cc1ncn(-c2ccc(Nc3cc(-c4nocc4C)ccn3)cc2)n1)c1'
[19:08:54] SMILES Parse Error: unclosed ring for input: 'O=C1NC(=O)c2c1c1ccccc1c2ccccc12'
[19:08:54] Can't kekulize mol.  Unkekulized atoms: 2 3 9 11 26
[19:08:54] Can't kekulize mol.  Unkekulized atoms: 11 12 14 15 17 18 19
[19:08:54] SMILES Parse Error: unclosed ring for input: 'CS(=O)(=O)N1CCc2nc(C3COc3ccccc3CNCc3ccco3)CC2C1'
[19:08:54] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 15 16 17 18
[19:08:54] SMILES Parse Error: unclosed ring for input: 'COc1ccc(C(=O)Nc2c3c(nn2-c2nc(C(C)(C)C)no2)CS(=O)(=O)C2)cc1OC'
[19:08:54] SMILES Parse Error: extra close parentheses while parsing: Cc1n

## 3. Utilize the trained LSTM model
Since training the lstm model has been completed, the trained model has been used to generate new smiles as follows.

In [ ]:
import sys 
import glob
sys.path.append('../src/model/')

for dataset in dataset_list:
	model_dir = f'{PRETRAIN_RESULTS}/{dataset}_results/vanilalstm'
	sample_num_list = [10000, 41743] # 41743 : numer of SMILES before randomization

	for sample_num in sample_num_list:
		outfd   = f'{PRETRAIN_RESULTS}/{dataset}_results/sampling_{sample_num}'
		appname = 'generative_models.apps.Sampling.SamplingApp'
		vocab   = f'{model_dir}/model/vocabulary.pickle'
		m_str 	= glob.glob(f'{model_dir}/model/best_model_structure_epoch*.pickle')[0]
		m_state = glob.glob(f'{model_dir}/model/best_model_epoch*.pth')[0]
		n 		= sample_num
		rseed 	= 42
		params = [	'--model=lstm',
					f'--model-structure={m_str}',
					f'--model-state={m_state}',
					f'--vocab={vocab}',
					f'--n={n}',
					'--allow-override',
					f'--outdir={outfd}',
					f'--random-seed={rseed}',
					f'--write-excel-file'
		]

		run(appname, *params)


[22:34:08] Can't kekulize mol.  Unkekulized atoms: 14 15 16 17 18 19 20 21 22
[22:34:08] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 9 15 16 17 18 19 20 25
[22:34:08] Can't kekulize mol.  Unkekulized atoms: 17 18 19 20 21
[22:34:08] Can't kekulize mol.  Unkekulized atoms: 4 5 6 14 15
[22:34:09] SMILES Parse Error: unclosed ring for input: 'COc1ccc(C)cc1N1C(=O)[C@H](c2ccccc2)C(c2n(c3ccc(Cl)cc2)c2ccccc2Cl)=NN1c1ccccc1'
[22:34:09] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 20
[22:34:09] SMILES Parse Error: unclosed ring for input: 'c1cc2occ(CCSc3ncnc4sccc35)n2c2c1'
[22:34:09] SMILES Parse Error: syntax error while parsing: CC[C@@H](CCc1ccccc1)NC(=O)NC[C@]1(c2cccc()C2)CC(C)C)c1
[22:34:09] SMILES Parse Error: Failed parsing SMILES 'CC[C@@H](CCc1ccccc1)NC(=O)NC[C@]1(c2cccc()C2)CC(C)C)c1' for input: 'CC[C@@H](CCc1ccccc1)NC(=O)NC[C@]1(c2cccc()C2)CC(C)C)c1'
[22:34:09] SMILES Parse Error: unclosed ring for input: 'COc1cccc([C@@H]2CCCN2c2nc3ccc4c(c3)OCCO4)cc1'
[22:34:09] Can't kekulize

## 4. Finetune the LSTM model
Finetuning the trained model using a small number of compounds is a routine by loading the trained model and training them using `finetune.py` code. The cli tool is `apps.FineTune.py`. 

To switch between **filtered** and **unfiltered** fine-tuning data, modify the variable `FINETUNE_FILTER` in `src/paths.py`.


In [3]:
from src.paths import FINETUNE_FILTER
print(f'Fine-tuning data status: {FINETUNE_FILTER}')

Fine-tuning data status: filtered


In [ ]:
import sys
import glob
sys.path.append('../src/model/')
from src.paths import ensure_dirs, FINETUNE_DATA, FINETUNE_RESULTS, FINETUNE_FILTER
ensure_dirs()

f_data_list = ['CHEMBL4005', 'CHEMBL1908389', 'CHEMBL284', 'CHEMBL214', 'CHEMBL253']

for dataset in dataset_list:
	for f_data in f_data_list:
		model_dir = f'{PRETRAIN_RESULTS}/{dataset}_results/vanilalstm'
		outfd     = f'{FINETUNE_RESULTS}/{dataset}_results/{f_data}_finetune'
		appname   = 'generative_models.apps.Finetune.FinetunerApp'
		vocab     = f'{model_dir}/model/vocabulary.pickle'
		m_str 	  = glob.glob(f'{model_dir}/model/best_model_structure_epoch*.pickle')[0]
		m_state   = glob.glob(f'{model_dir}/model/best_model_epoch*.pth')[0]
		save_epoch_models = False

		data      = f'{FINETUNE_DATA}/{FINETUNE_FILTER}-{f_data}_train_rdsmi3.tsv'
		smicol    = 'rd3_smiles'
		epochs	  = 100
		rseed 	  = 42
		batchsize = 16
		lr 		  = 1e-4 # small learning rate
		sampling_epoch = 10000

		params = [	
					f'--data={data}',
					f'--smi-colname={smicol}',
					f'--sampling-epoch={sampling_epoch}',
					'--model=lstm',
					f'--model-structure={m_str}',
					f'--model-state={m_state}',
					f'--vocab={vocab}',
					f'--epochs={epochs}',
					'--override-folder=1',
					f'--outdir={outfd}',
					f'--random-seed={rseed}',
					f'--batch-size={batchsize}',
					f'--lr={lr}',
					f'--exclude-pad-loss=1',
					'--use-cpus' # force working on CPUs
		]

		run(appname, *params)


Start logging
Starting: Finetuner, Namespace(num_workers=1, outdir='/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/results/finetune/filtered/pubchem_filtered_ac_results/CHEMBL4005_finetune', case='', override_folder=1, tensorboard_prefix='tensor_board', data='/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/data/finetune/filtered/filtered_CHEMBL4005_train_rdsmi3.tsv', smi_colname='rd3_smiles', vocab='/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/results/pretrain/pubchem_filtered_ac_results/vanilalstm/model/vocabulary.pickle', random_seed=42, debug=None, sampling_epoch=10000, model='lstm', write_xlsx=None, save_snapshot_models='', batch_size=16, epochs=100, validation_ratio=0.1, lr=0.0001, exclude_pad_loss=1, early_stopping_patience=0, load_model=None, model_structure='/home/abe/Paper/Pretraining-Assesment-for-LSTM-Molecular-Generation/results/pretrain/pubchem_filtered_ac_results/vanilalstm/model/best_model_structure_epoch28.

[01:02:55] SMILES Parse Error: unclosed ring for input: 'CCOc1ccc(NC(=O)C2=C(CCNC(=O)C2CC2)N(C)C)cc1'
[01:02:55] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 23
[01:02:55] SMILES Parse Error: unclosed ring for input: 'COc1ccc(-c2nc(C#N)c(N3CCC3(CC3)OCCO4)c2C)cc1OC'
[01:02:55] SMILES Parse Error: unclosed ring for input: 'N#Cc1cc2c(nc1SCc1cnn3n1Cc1ccccc1)CCCC2'
[01:02:55] SMILES Parse Error: unclosed ring for input: 'CN(C)C(=O)N1CCN(Cc2nc3sc(-c4ccccc4)c(=O)[nH]2)CC1'
[01:02:55] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 17
[01:02:55] SMILES Parse Error: ring closure 3 duplicates bond between atom 14 and atom 15 for input: 'Clc1ccc(C23CCN(Cc3nnnn3C3CCCCC3)CC2)cc1'
[01:02:55] SMILES Parse Error: unclosed ring for input: 'C[C@H]1CN([C@@H](C)CO)C(=O)Cc2cc(NC(=O)Nc3ccccc3)ccc2O[C@H](C)CN1CCCC1'
[01:02:55] Can't kekulize mol.  Unkekulized atoms: 14 15 16 17 18 20 21 23 24
[01:02:55] Explicit valence for atom # 10 N, 4, is greater than permitted
[01:02:55] Can't kekulize m

sampling smiles at most 1000.


[01:02:57] SMILES Parse Error: unclosed ring for input: 'COc1ccccc1C1(CCn2c(=O)oc2ccccc2)CCO1'
[01:02:57] SMILES Parse Error: extra open parentheses for input: 'Cc1ccc(N(C2=NC3CCCCC3=NC2(C(=O)NCc2ccco2)C2)cc1'
[01:02:57] Can't kekulize mol.  Unkekulized atoms: 6 7 17 18 19 20 21 22 23
[01:02:57] SMILES Parse Error: unclosed ring for input: 'COc1ccccc1N1CCN(CCC(Oc2ccc(C)cc2)CC2CCO)CC1'
[01:02:57] Can't kekulize mol.  Unkekulized atoms: 6 7 8
[01:02:57] Can't kekulize mol.  Unkekulized atoms: 15 16 17
[01:02:57] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[01:02:57] Can't kekulize mol.  Unkekulized atoms: 12 13 24
[01:02:57] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
[01:02:57] Can't kekulize mol.  Unkekulized atoms: 4 5 6 11 12 13 14 15 16
[01:02:57] SMILES Parse Error: unclosed ring for input: 'O=C(CN1CCCC1)N1c2ccccc2C1(c1ccccc1)CCCC1'
[01:02:57] SMILES Parse Error: unclosed ring for input: 'CC(C)N1CC2C3CC1C(C)(CO)N4C(=O)c1cc2ccccc2o1'
[01:02:57] Can't kekulize mol.  Unkek

Epoch 1/100, training: 9 batches, size 16*1
Epoch 1 -- Batch 1/ 9, training loss 1.9817802906036377
Epoch 1 -- Batch 2/ 9, training loss 1.890822172164917
Epoch 1 -- Batch 3/ 9, training loss 1.9888304471969604
Epoch 1 -- Batch 4/ 9, training loss 1.689042091369629
Epoch 1 -- Batch 5/ 9, training loss 1.3676780462265015
Epoch 1 -- Batch 6/ 9, training loss 1.5268007516860962
Epoch 1 -- Batch 7/ 9, training loss 1.4545291662216187
Epoch 1 -- Batch 8/ 9, training loss 1.271896243095398
Epoch 1 -- Batch 9/ 9, training loss 1.192453384399414
----------------------------------------------------------------------


KeyboardInterrupt: 